In [1]:
import numpy as np
import random

In [42]:
class HMM():
    def __init__(self,no_of_state,no_of_value,states,values):
        self.emssion_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_value)])]) #b[i][O]  --> probability to emit values[O] from states[i]
        
        self.initial_matrix= np.random.rand(no_of_state) #π[i] --> probability to start at states[i]
        self.initial_matrix/= self.initial_matrix.sum()
        
        self.transition_matrix=np.array([ran/ran.sum() for ran in np.array([np.random.rand(no_of_state) for i in range(no_of_state)])])  #a[i][j] --> probability from states[i] transit to states[j]
        self.states=states
        self.values=values
    def debug(self):
        print('initial_matrix\n',self.initial_matrix)
        print('transition_matrix\n',self.transition_matrix)
        print('emssion_matrix\n',self.emssion_matrix)

In [43]:
hmm=HMM(3,3,['hot','cold','warm'],['eat ice cream','hotpot','sleep'])

In [44]:
hmm.debug()

initial_matrix
 [0.25315538 0.35188802 0.3949566 ]
transition_matrix
 [[0.42954679 0.28879454 0.28165867]
 [0.3783173  0.26951259 0.35217011]
 [0.35490393 0.29520606 0.34989002]]
emssion_matrix
 [[0.46566151 0.36672883 0.16760966]
 [0.47029498 0.35214097 0.17756405]
 [0.24369216 0.24460942 0.51169842]]
